In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e12/sample_submission.csv
/kaggle/input/playground-series-s4e12/train.csv
/kaggle/input/playground-series-s4e12/test.csv


In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
from sklearn.model_selection import ParameterSampler, KFold

import warnings
warnings.filterwarnings("ignore")

import torch

def check_gpu_status():
    if torch.cuda.is_available():
        print(f"可用GPU数量: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
            print(f"显存使用情况: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MB 已用, {torch.cuda.memory_reserved(i) / 1024**2:.2f} MB 总分配")
            print(f"显存空余: {torch.cuda.memory_reserved(i) - torch.cuda.memory_allocated(i):.2f} 字节\n")
    else:
        print("未检测到 GPU！")

check_gpu_status()

可用GPU数量: 2
GPU 0: Tesla T4
显存使用情况: 0.00 MB 已用, 0.00 MB 总分配
显存空余: 0.00 字节

GPU 1: Tesla T4
显存使用情况: 0.00 MB 已用, 0.00 MB 总分配
显存空余: 0.00 字节



In [3]:
train = pd.read_csv("/kaggle/input/playground-series-s4e12/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s4e12/test.csv")

sample = pd.read_csv('/kaggle/input/playground-series-s4e12/sample_submission.csv')

In [4]:
train.head()

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [5]:
def date(Df):

    Df['Policy Start Date'] = pd.to_datetime(Df['Policy Start Date'])
    Df['Year'] = Df['Policy Start Date'].dt.year
    Df['Day'] = Df['Policy Start Date'].dt.day
    Df['Month'] = Df['Policy Start Date'].dt.month
    Df['Month_name'] = Df['Policy Start Date'].dt.month_name()
    Df['Day_of_week'] = Df['Policy Start Date'].dt.day_name()
    Df['Week'] = Df['Policy Start Date'].dt.isocalendar().week
    Df['Year_sin'] = np.sin(2 * np.pi * Df['Year'])
    Df['Year_cos'] = np.cos(2 * np.pi * Df['Year'])
    Df['Month_sin'] = np.sin(2 * np.pi * Df['Month'] / 12) 
    Df['Month_cos'] = np.cos(2 * np.pi * Df['Month'] / 12)
    Df['Day_sin'] = np.sin(2 * np.pi * Df['Day'] / 31)  
    Df['Day_cos'] = np.cos(2 * np.pi * Df['Day'] / 31)
    Df['Group']=(Df['Year']-2020)*48+Df['Month']*4+Df['Day']//7
    
    Df.drop('Policy Start Date', axis=1, inplace=True)

    return Df
train = date(train)
test = date(test)

In [6]:
train.head()

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Month_name,Day_of_week,Week,Year_sin,Year_cos,Month_sin,Month_cos,Day_sin,Day_cos,Group
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,December,Saturday,51,-6.447061e-13,1.0,-2.449294e-16,1.000000e+00,-0.998717,-0.050649,195
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,June,Monday,24,-6.447061e-13,1.0,1.224647e-16,-1.000000e+00,0.651372,-0.758758,169
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,September,Saturday,39,-6.447061e-13,1.0,-1.000000e+00,-1.836970e-16,-0.201299,0.979530,184
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,June,Wednesday,24,1.585375e-14,1.0,1.224647e-16,-1.000000e+00,0.651372,-0.758758,217
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,December,Wednesday,48,-1.468363e-13,1.0,-2.449294e-16,1.000000e+00,0.201299,0.979530,96


In [7]:
test.head()

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Month_name,Day_of_week,Week,Year_sin,Year_cos,Month_sin,Month_cos,Day_sin,Day_cos,Group
0,1200000,28.0,Female,2310.0,NaN,4.0,Bachelor's,Self-Employed,7.657981,Rural,...,June,Sunday,22,-6.447061e-13,1.0,1.224647e-16,-1.000000,0.724793,0.688967,168
1,1200001,31.0,Female,126031.0,Married,2.0,Master's,Self-Employed,13.381379,Suburban,...,April,Monday,17,1.585375e-14,1.0,8.660254e-01,-0.500000,-0.968077,-0.250653,211
2,1200002,47.0,Female,17092.0,Divorced,0.0,PhD,Unemployed,24.354527,Urban,...,April,Wednesday,14,-6.447061e-13,1.0,8.660254e-01,-0.500000,0.848644,0.528964,160
3,1200003,28.0,Female,30424.0,Divorced,3.0,PhD,Self-Employed,5.136225,Suburban,...,October,Wednesday,43,-6.447061e-13,1.0,-8.660254e-01,0.500000,-0.937752,0.347305,187
4,1200004,24.0,Male,10863.0,Divorced,2.0,High School,Unemployed,11.844155,Suburban,...,November,Friday,47,-1.468363e-13,1.0,-5.000000e-01,0.866025,-0.848644,0.528964,95


In [8]:
class CategoricalEncoder:
    def __init__(self, train, test):
        self.train = train
        self.test = test

    def frequency_encode(self, cat_cols, feature_cols, drop_org=False):
        combined = pd.concat([self.train, self.test], axis=0, ignore_index=True)

        new_cat_cols = [] 
        for col in cat_cols:
            freq_encoding = combined[col].value_counts().to_dict()
            
            self.train[f"{col}_freq"] = self.train[col].map(freq_encoding).astype('float')
            self.test[f"{col}_freq"] = self.test[col].map(freq_encoding).astype('float')

            new_col_name = f"{col}_freq"
            new_cat_cols.append(new_col_name)
            feature_cols.append(new_col_name)
            if drop_org:
                feature_cols.remove(col)

        return self.train, self.test, new_cat_cols, feature_cols

In [9]:
cat_cols = [col for col in train.columns if train[col].dtype == 'object']
feature_cols = list(test.columns)

encoder = CategoricalEncoder(train, test)
train, test, cat_cols, feature_cols = encoder.frequency_encode(cat_cols, feature_cols, drop_org=True)

In [10]:
train = train[feature_cols + ['Premium Amount']]
test = test[feature_cols]
X = train.drop('Premium Amount', axis=1)  
y = train['Premium Amount']

In [11]:
X.head()

,id,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Year,...,Education Level_freq,Occupation_freq,Location_freq,Policy Type_freq,Customer Feedback_freq,Smoking Status_freq,Exercise Frequency_freq,Property Type_freq,Month_name_freq,Day_of_week_freq
0,0,19.0,10049.0,1.0,22.598761,2.0,17.0,372.0,5.0,2023,...,505457.0,470636.0,663201.0,669475.0,625952.0,996268.0,510693.0,667500.0,162307.0,284861.0
1,1,39.0,31678.0,3.0,15.569731,1.0,12.0,694.0,2.0,2023,...,506370.0,NaN,668067.0,665822.0,629122.0,1003732.0,498230.0,667500.0,164442.0,287191.0
2,2,23.0,25602.0,3.0,47.177549,1.0,14.0,NaN,3.0,2023,...,482198.0,470636.0,668732.0,669475.0,614826.0,1003732.0,510693.0,667500.0,165556.0,284861.0
3,3,21.0,141855.0,2.0,10.938144,1.0,0.0,367.0,1.0,2024,...,505457.0,NaN,668067.0,664703.0,625952.0,1003732.0,491143.0,666022.0,164442.0,287424.0
4,4,21.0,39651.0,1.0,20.376094,0.0,8.0,598.0,4.0,2021,...,505457.0,470636.0,668067.0,669475.0,625952.0,1003732.0,510693.0,667500.0,162307.0,287424.0


# Evaluation

In [12]:
y_log = np.log1p(y)

In [13]:
def rmsle(y_true, y_pred):
    return np.sqrt(np.mean(np.square(np.log1p(y_true) - np.log1p(y_pred))))

In [14]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold



# 定义 Optuna 的目标函数
def objective(trial, X, y, n_splits=5, random_state=42):
    # 定义超参数搜索空间
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 1000, 3000, step=1000),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.01, 0.03, 0.05]),
        'max_depth': trial.suggest_int('max_depth', 4, 10, step=2),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0, step=0.1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0, step=0.1),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'gamma': trial.suggest_float('gamma', 0, 0.3, step=0.1),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 0.1, step=0.01),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1.0, step=0.1),
    }

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    oof = np.zeros(len(X))
    fold_scores = []

    for fold, (train_idx, valid_idx) in enumerate(kf.split(X)):
        print(f"  Fold {fold + 1}")
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = np.log1p(y.iloc[train_idx]), np.log1p(y.iloc[valid_idx])

        model = xgb.XGBRegressor(
            **param,
            objective='reg:squarederror',
            random_state=random_state,
            eval_metric="rmse",
            verbosity=0,
            tree_method='gpu_hist',  # 使用 GPU 加速
            booster='gbtree',
        )
        
        model.fit(
            X_train, y_train,
            eval_set=[(X_valid, y_valid)],
            early_stopping_rounds=300,
            verbose=False,
        )

        # 预测和评估
        oof[valid_idx] = np.maximum(0, model.predict(X_valid))
        fold_rmsle = rmsle(np.expm1(y_valid), np.expm1(oof[valid_idx]))
        fold_scores.append(fold_rmsle)

        print(f"    Fold {fold + 1} RMSLE: {fold_rmsle:.4f}")

    mean_score = np.mean(fold_scores)
    print(f"Mean RMSLE: {mean_score:.4f}")
    
    return mean_score

# 使用 Optuna 进行超参数优化
def optimize_hyperparameters(X, y, n_trials=2, n_splits=5):
    study = optuna.create_study(direction='minimize')  # 目标是最小化 RMSLE
    study.optimize(lambda trial: objective(trial, X, y, n_splits), n_trials=n_trials)
    
    print("\nBest Parameters:", study.best_params)
    print(f"Best RMSLE: {study.best_value:.4f}")
    return study.best_params, study.best_value, study

# 执行超参数优化
best_params, best_rmsle, study = optimize_hyperparameters(X, y)

# 使用最佳参数训练最终模型
kf = KFold(n_splits=5, shuffle=True, random_state=42)
models = []
for fold, (train_idx, valid_idx) in enumerate(kf.split(X)):
    print(f"Final Model Training - Fold {fold + 1}")
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = np.log1p(y.iloc[train_idx]), np.log1p(y.iloc[valid_idx])

    model = xgb.XGBRegressor(
        **best_params,
        objective='reg:squarederror',
        random_state=42,
        eval_metric="rmse",
        tree_method='gpu_hist',
        booster='gbtree',
    )
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        early_stopping_rounds=300,
        verbose=False,
    )
    models.append(model)

# 对测试集进行预测
test_predictions = np.zeros(len(test))
for model in models:
    test_predictions += np.maximum(0, np.expm1(model.predict(test))) / len(models)

# 将预测结果保存到 CSV 文件
sample['Premium Amount'] = test_predictions
sample.to_csv('submission_xgb_optuna.csv', index=False)

[I 2024-12-20 04:00:08,908] A new study created in memory with name: no-name-fef8497d-db05-4a30-8024-e6c77c9000bf


  Fold 1
    Fold 1 RMSLE: 1.0464
  Fold 2
    Fold 2 RMSLE: 1.0456
  Fold 3
    Fold 3 RMSLE: 1.0464
  Fold 4
    Fold 4 RMSLE: 1.0445
  Fold 5


[I 2024-12-20 04:00:59,993] Trial 0 finished with value: 1.045757219088634 and parameters: {'n_estimators': 3000, 'learning_rate': 0.05, 'max_depth': 8, 'subsample': 0.7, 'colsample_bytree': 1.0, 'min_child_weight': 1, 'gamma': 0.0, 'reg_alpha': 0.08, 'reg_lambda': 0.6000000000000001}. Best is trial 0 with value: 1.045757219088634.


    Fold 5 RMSLE: 1.0460
Mean RMSLE: 1.0458
  Fold 1
    Fold 1 RMSLE: 1.0489
  Fold 2
    Fold 2 RMSLE: 1.0476
  Fold 3
    Fold 3 RMSLE: 1.0486
  Fold 4
    Fold 4 RMSLE: 1.0469
  Fold 5


[I 2024-12-20 04:02:04,446] Trial 1 finished with value: 1.048041045748685 and parameters: {'n_estimators': 1000, 'learning_rate': 0.03, 'max_depth': 4, 'subsample': 0.7999999999999999, 'colsample_bytree': 0.7999999999999999, 'min_child_weight': 1, 'gamma': 0.1, 'reg_alpha': 0.01, 'reg_lambda': 0.2}. Best is trial 0 with value: 1.045757219088634.


    Fold 5 RMSLE: 1.0482
Mean RMSLE: 1.0480

Best Parameters: {'n_estimators': 3000, 'learning_rate': 0.05, 'max_depth': 8, 'subsample': 0.7, 'colsample_bytree': 1.0, 'min_child_weight': 1, 'gamma': 0.0, 'reg_alpha': 0.08, 'reg_lambda': 0.6000000000000001}
Best RMSLE: 1.0458
Final Model Training - Fold 1
Final Model Training - Fold 2
Final Model Training - Fold 3
Final Model Training - Fold 4
Final Model Training - Fold 5
